In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-189942
Azure region: southcentralus
Subscription id: aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee
Resource group: aml-quickstarts-189942


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "canina-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('This compute target already exists.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",
                                                          max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    print('A new compute target has been created.')
    
compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

A new compute target has been created.
InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "C": uniform(0.05, 10),
    "max_iter": choice(100, 300, 500, 700, 1000)
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=3)

if "training" not in os.listdir():
    os.mkdir("./training")

#### Copy train.py script into "./training" to use it in ScriptRunConfig object.
import shutil

shutil.copy('train.py', "./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./training',
                     script='train.py',
                     compute_target=compute_target,
                     environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=4,
                                    max_concurrent_runs=4)

# SKIP

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "C": uniform(0.05, 10),
    "max_iter": choice(100, 300, 500, 700, 1000)
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=3)

if "training" not in os.listdir():
    os.mkdir("./training")

#### Copy train.py script into "./training" to use it in ScriptRunConfig object.
import shutil

shutil.copy('train.py', "./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./training',
                     script='train.py',
                     compute_target=compute_target,
                     environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=24,
                                    max_concurrent_runs=4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)

In [5]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [6]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
# We need to wait before retrieving the best model
hyperdrive_run.wait_for_completion(show_output=True)

best_run_hyperdrive = hyperdrive_run.get_best_run_by_primary_metric()
# Print best metrics 
best_metrics = best_run_hyperdrive.get_metrics()
print('Run Id : ', best_run_hyperdrive.id)
print('Metrics :',best_metrics)

# save the model
best_hyperdrive_model = best_run_hyperdrive.register_model(model_name = 'hyperdrivemodel', model_path = 'outputs/model.joblib')

RunId: HD_1226ca73-fbfe-4afa-bbee-12d1798f48e7
Web View: https://ml.azure.com/runs/HD_1226ca73-fbfe-4afa-bbee-12d1798f48e7?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-189698/workspaces/quick-starts-ws-189698&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Execution Summary
RunId: HD_1226ca73-fbfe-4afa-bbee-12d1798f48e7
Web View: https://ml.azure.com/runs/HD_1226ca73-fbfe-4afa-bbee-12d1798f48e7?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-189698/workspaces/quick-starts-ws-189698&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Run Id :  HD_1226ca73-fbfe-4afa-bbee-12d1798f48e7_23
Metrics : {'Regularization Strength:': 6.463739391157785, 'Max iterations:': 300, 'Accuracy': 0.9095599393019727}


In [7]:
# This print is to see the best model
print(best_hyperdrive_model.name, best_hyperdrive_model.id, best_hyperdrive_model.version, sep='\t')

hyperdrivemodel	hyperdrivemodel:1	1


In [9]:
best_run_hyperdrive.get_file_names()

['logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/rslex.log',
 'outputs/model.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/lifecycler/vm-bootstrapper.log',
 'user_logs/std_log.txt']

In [13]:
best_run_hyperdrive.download_file(best_run_hyperdrive.get_file_names()[3],
                                 output_file_path = './outputs/')

In [14]:
joblib.load('./outputs/model.joblib')

LogisticRegression(C=6.463739391157785, max_iter=300)

In [4]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
path_data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path_data, validate=True, include_path=False, infer_column_types=True, set_column_types=None, separator=',', header=True, partition_format=None, support_multi_line=False, empty_as_string=False, encoding='utf8')
    

In [5]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [6]:
ds_ = x.join(y)

In [8]:
from azureml.core.dataset import Dataset

ds_final = Dataset.from_pandas_dataframe(ds_)

"Dataset.from_pandas_dataframe" is deprecated after version 1.0.69. Please use "Dataset.Tabular.register_pandas_dataframe" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


In [11]:
x_ = Dataset.from_pandas_dataframe(x)
y_ = Dataset.from_pandas_dataframe(y)

In [23]:
import pandas as pd

x_df = pd.read_csv('x.csv')
y_df = pd.read_csv('y.csv')

In [25]:
from azureml.core import Datastore
datastore = Datastore.get(ws, 'workspaceblobstore')
dataset_ = Dataset.Tabular.register_pandas_dataframe(x_df, datastore, "Xdataset", show_progress=True)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/c8862439-ab1c-4fc8-a8fc-673b4215bfb5/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [26]:
dataset_y = Dataset.Tabular.register_pandas_dataframe(y_df, datastore, "Ydataset", show_progress=True)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/24703242-52b4-49f5-85b3-10cefb0b5169/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [27]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_settings = {
    "experiment_timeout_minutes" : 30,
    "enable_early_stopping" : True,
    "primary_metric" : 'accuracy',
    "featurization" : 'auto',
    "verbosity" : logging.INFO,
    "n_cross_validations" : 5    
}

# Split data into train and test sets using parameter test_size.

automl_config = AutoMLConfig(
    task='classification',
    debug_log='automl_errors.log',
    compute_target=compute_target,
    X=dataset_,
    y=dataset_y,
    test_size=0.2,
    **automl_settings)

In [28]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on canina-cluster with default configuration
Running on remote compute: canina-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_56869b41-f381-4f1b-913a-decf2ef27f54,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Nu

In [29]:
automl_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_56869b41-f381-4f1b-913a-decf2ef27f54,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|2954                          |1                               |26360                                 |
+------------------------------+--------------------------------+----

{'runId': 'AutoML_56869b41-f381-4f1b-913a-decf2ef27f54',
 'target': 'canina-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-03-25T04:15:02.762655Z',
 'endTimeUtc': '2022-03-25T04:47:25.441854Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'canina-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee","resource_group":"aml-quickstarts-189942","workspace_name":"quick-starts-ws-189942","region":"southcentralus","compute_target":"canina-c

In [30]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###



best_run_automl, best_automl_model = automl_run.get_output()

automl_model_name = best_run_automl.properties["model_name"]
registered_model = automl_run.register_model(model_name=automl_model_name,
                                                 description = "Automl model",
                                                 tags=None)


In [40]:
best_run_automl.get_file_names()[40]

'outputs/model.pkl'

In [38]:
best_run_automl.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'confusion_matrix',
 'explanation/d5808861/classes.interpret.json',
 'explanation/d5808861/expected_values.interpret.json',
 'explanation/d5808861/features.interpret.json',
 'explanation/d5808861/global_names/0.interpret.json',
 'explanation/d5808861/global_rank/0.interpret.json',
 'explanation/d5808861/global_values/0.interpret.json',
 'explanation/d5808861/local_importance_values.interpret.json',
 'explanation/d5808861/per_class_names/0.interpret.json',
 'explanation/d5808861/per_class_rank/0.interpret.json',
 'explanation/d5808861/per_class_values/0.interpret.json',
 'explanation/d5808861/rich_metadata.interpret.json',
 'explanation/d5808861/true_ys_viz.interpret.json',
 'explanation/d5808861/visualization_dict.interpret.json',
 'explanation/d5808861/ys_pred_proba_viz.interpret.json',
 'explanation/d5808861/ys_pred_viz.interpret.json',
 'explanation/fa3efe32/classes.interpret.json',
 'explanation/fa3efe32/eval_data_viz.interpret.json',
 'expl

In [43]:
best_run_automl.download_file(best_run_automl.get_file_names()[40],
                                 output_file_path = './outputs/')

In [54]:
test_run_automl

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,3b6d21eb-5711-4857-9041-9e8982e8210e,automl.model_test,Failed,Link to Azure Machine Learning studio,Link to Documentation


In [50]:
test_run_automl = next(best_run_automl.get_children(type='automl.model_test'))
#test_run_automl.wait_for_completion(show_output=False, wait_post_processing=True)

In [51]:
# Test the performance of the automl model.
test_run_metrics = test_run_automl.get_metrics()
for m, val in test_run_metrics.items():
    print(f"{m}: {val}")

In [52]:
test_run_metrics

{}

### Delete the compute cluster "canina-cluster"

In [ ]:
# Remove the compute cluster following training.
compute_target.delete()